# Partido Diablos vs Sultanes 2025-06-17

Notebook para analizar y generar reportes del partido entre Diablos Rojos y Sultanes de Monterrey del 17 de junio de 2025. Se utilizan datos de TrackMan y la API de MLB.

* [TrackMan](https://support.trackmanbaseball.com/hc/en-us/articles/5089413493787-V3-FAQs-Radar-Measurement-Glossary-Of-Terms)
* [MLB](https://github.com/MajorLeagueBaseball/google-cloud-mlb-hackathon/tree/main/datasets/mlb-statsapi-docs)

## Importar bibliotecas y datos

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns

In [ ]:
ruta = '../data/20250617-EstadioAlfredo-1.csv'
df = pd.read_csv(ruta, sep=',', encoding='utf-8')
#df.head()

In [ ]:
ruta = '../data/pitchers.csv'
pitchers = pd.read_csv(ruta, sep=',', encoding='utf-8')
#pitchers.head()

In [ ]:
ruta = '../data/bateadores.csv'
bateadores = pd.read_csv(ruta, sep=',', encoding='utf-8')
#bateadores.head()

## Funciones

In [ ]:
def graficarZonaStrike(ax=None, sz_top=3.5, sz_bot=1.5, vistaCatcher=True):
    if ax is None:
        fig, ax = plt.subplots(figsize=(5, 6))
    sz_left = -0.708
    sz_right = 0.708
    ax.plot([sz_left, sz_right], [sz_bot, sz_bot], 'k-')
    ax.plot([sz_left, sz_left], [sz_bot, sz_top], 'k-')
    ax.plot([sz_right, sz_right], [sz_bot, sz_top], 'k-')
    ax.plot([sz_left, sz_right], [sz_top, sz_top], 'k-')

    if vistaCatcher:
        ax.plot([sz_left, sz_right], [0, 0], 'k-')
        ax.plot([sz_left, sz_left], [0, -0.15], 'k-')
        ax.plot([sz_right, sz_right], [0, -0.15], 'k-')
        ax.plot([sz_left, 0], [-0.15, -0.4], 'k-')
        ax.plot([sz_right, 0], [-0.15, -0.4], 'k-')
    else:
        ax.plot([0, sz_left], [0, -0.25], 'k-')
        ax.plot([0, sz_right], [0, -0.25], 'k-')
        ax.plot([sz_left, sz_left], [-0.25, -0.4], 'k-')
        ax.plot([sz_right, sz_right], [-0.25, -0.4], 'k-')
        ax.plot([sz_left, sz_right], [-0.4, -0.4], 'k-')

    ax.set_xlim(-2.7, 2.7)
    ax.set_ylim(-0.5, 5.7)

    sns.set_style('white')
    sns.despine(right=True, top=True, left=True, bottom=True)
    ax.set_xticks([])
    ax.set_yticks([])
    
    return ax

In [ ]:
def graficarDistribucionLanzamientos(dfDist, dfPos, titulo):
    colores = {'FF': '#005F73', 'Curveball': '#94D2BD', 'Splitter': '#EE9B00', 'Slider': '#BB3E03'}
    tipos = dfDist['Tipo lanzamiento'].unique().tolist()
    nTipos = len(tipos)
    columns = 5
    rows = (nTipos//columns) + 1
    fig = plt.figure(figsize=(columns * 3, rows * 3))
    plt.tight_layout()
    gs = GridSpec(rows, columns, figure=fig, 
                width_ratios=[1.7] + [1]*(columns-1), 
                height_ratios=[1]*rows,
                wspace=0.05, hspace=0.05,
                top=0.87
                )
    
    ax1 = fig.add_subplot(gs[0, 0])
    ax1_pos = ax1.get_position()
    ax1.set_position([ax1_pos.x0, ax1_pos.y0 + 0.3, ax1_pos.width, ax1_pos.height])
    plot = ax1.pie(
        dfDist['Porcentaje'], 
        autopct='%1.1f%%', startangle=90,
        colors=dfDist['Tipo lanzamiento'].map(colores).to_list(),
        wedgeprops=dict(width=0.3),
        radius=0.7,
        pctdistance=1.4
    )
    ax1.text(
        0, 0,
        dfDist['Total'].sum(),
        ha='center', va='center',
        fontsize=14, weight='bold'
    )
    ax1.legend(plot[0], dfDist['Tipo lanzamiento'],
          loc="lower center",
          ncol=3,
          frameon=False,
          fontsize=12,
          bbox_to_anchor=(0.5, -0.2)
    )
    
    for i in range(rows):
        for j in range(1, columns):
            if (i * columns + j - 1) >= nTipos:
                break
            tipo = tipos[(columns-1)*(i)+(j-1)]
            ax = fig.add_subplot(gs[i, j])
            graficarZonaStrike(ax=ax)
            try:
                sns.kdeplot(
                    data=dfPos[dfPos['TaggedPitchType'] == tipo],
                    x=-dfPos['PlateLocSide'],
                    y='PlateLocHeight',
                    color='red',
                    fill=True
                )
            except:
                pass
            sns.scatterplot(
                data=dfPos[dfPos['TaggedPitchType'] == tipo],
                x=-dfPos['PlateLocSide'],
                y='PlateLocHeight'
            )
            ax.set_title(tipo, fontsize=12, y=-0.1)

    fig.suptitle(titulo, fontsize=16, y=0.95)
    fig.subplots_adjust(left=0.05, right=0.95, bottom=0.05, top=0.9)
    return fig

In [ ]:
def graficarSituacionPitcher(df, total, titulo):
    colores = {'Ponche': '#001219', 'Base por bolas': '#005F73', 'Base por golpe': '#0A9396', 
               'Hit': '#94D2BD', 'Out': '#E9D8A6', 'Error': '#EE9B00', 'Bola ocupada': '#CA6702', 
               'Sacrificio': '#BB3E03', 'Otro': '#9B2226'}
    fig, ax = plt.subplots(figsize=(5.1, 3))
    plot = ax.pie(
        df['Porcentaje'], 
        startangle=90,
        colors=df['index'].map(colores).to_list(),
        wedgeprops=dict(width=0.3),
        radius=0.7,
        pctdistance=1.4,
        labels=None
    )
    ax.text(
        0, 0,
        total,
        ha='center', va='center',
        fontsize=12, weight='bold'
    )
    ax.legend(
        plot[0], 
        [f"{l} ({p:.1f}%)" for l, p in zip(df['index'], df['Porcentaje'])],
        loc="lower center",
        ncol=2,
        frameon=False,
        fontsize='small'
    )

    fig.suptitle(titulo, fontsize=16, y=0.95, x=0.2)
    plt.subplots_adjust(left=0.3)

    plt.close()

    return fig

In [ ]:
def getZona(x, y, sz_top=3.5, sz_bot=1.5):
    sz_right = 0.708
    sz_left = -sz_right
    sz_midle = (sz_top - sz_bot) / 2 + sz_bot
    hor_delta = (sz_right * 2) / 3
    vert_delta = (sz_top - sz_bot) / 3

    if ((y >= sz_top and x < 0) or (y >= sz_midle and x < sz_left)): return 11
    if ((y >= sz_top and x >= 0) or (y >= sz_midle and x >= sz_right)): return 12
    if ((y < sz_bot and x < 0) or (y < sz_midle and x < sz_left)): return 13
    if ((y < sz_bot and x >= 0) or (y < sz_midle and x >= sz_right)): return 14

    if (y >= (sz_top - vert_delta)): zona = 0
    elif (y >= (sz_bot + vert_delta)): zona = 3
    else: zona = 6

    if (x < (sz_left + hor_delta)): zona += 1
    elif (x < (sz_right - hor_delta)): zona += 2
    else: zona += 3
    return zona

## Exploracion inicial de datos

In [ ]:
df.shape

In [ ]:
list(df.columns)

In [ ]:
df.dtypes

In [ ]:
df.describe()

## Limpiar datos

In [ ]:
# Elimiar columnas que no se van a usar
df = df.drop(columns=[
    'UTCDate', 'UTCTime', 'LocalDateTime', 'UTCDateTime', 'PitcherId', 'PitcherTeam', 'BatterId',
    'BatterTeam', 'PitcherSet', 'AutoPitchType', 'AutoHitType', 'Notes', 'Distance', 'Stadium',
    'HomeTeamForeignID', 'AwayTeamForeignID', 'GameForeignID', 'Level', 'League', 'GameID',
    'PitchUID', 'System'
  ], axis=1)

In [ ]:
df['Pitcher'] = df['Pitcher'].str.split(', ').apply(lambda x: x[1] + ' ' + x[0])
df['Batter'] = df['Batter'].str.split(', ').apply(lambda x: x[1] + ' ' + x[0])
df['TaggedPitchType'] = df['TaggedPitchType'].str.replace('FourSeamFastBall', 'FF', regex=True)

df['HomeScore'] = np.where(df['Top/Bottom'] == 'Bottom', df['RunsScored'], 0)
df['AwayScore'] = np.where(df['Top/Bottom'] == 'Top', df['RunsScored'], 0)
df['HomeScore'] = df['HomeScore'].cumsum().shift(1).fillna(0)
df['AwayScore'] = df['AwayScore'].cumsum().shift(1).fillna(0)
df['RunDif'] = np.where(df['Top/Bottom'] == 'Top', df['HomeScore'] - df['AwayScore'], df['AwayScore'] - df['HomeScore'])
df['PitchZone'] = df.apply(lambda row: getZona(row['PlateLocSide'], row['PlateLocHeight']), axis=1)
df['PAResult'] = np.where(
    df['KorBB'] == 'Strikeout', 'Ponche',
    np.where(df['KorBB'] == 'Walk', 'Base por bolas',
    np.where(df['PitchCall'] == 'HitByPitch', 'Base por golpe', 
    np.where((df['PlayResult'] == 'Single') |
             (df['PlayResult'] == 'Double') |
             (df['PlayResult'] == 'Triple') |
             (df['PlayResult'] == 'HomeRun'), 'Hit',
    np.where(df['PlayResult'] == 'Out', 'Out',
    np.where(df['PlayResult'] == 'Error', 'Error', 
    np.where(df['PlayResult'] == 'FieldersChoice', 'Bola ocupada', 
    np.where(df['PlayResult'] == 'Sacrifice', 'Sacrificio', pd.NA)
    ))))))
)
df['PAResult'] = df.groupby(['Inning', 'Top/Bottom', 'PAofInning'])['PAResult'].transform('last')

In [ ]:
pitchers['Pitcher'] = pitchers['Pitcher'].str.replace('Á', 'A').str.replace('É', 'E').str.replace('Í', 'I').str.replace('Ó', 'O').str.replace('Ú', 'U')
pitchers['Pitcher'] = pitchers['Pitcher'].str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')
pitchers['InningsPitch'] = pitchers['TotalOuts'].apply(lambda x: f'{(x // 3)}.{(x % 3)}')

In [ ]:
bateadores['Bateador'] = bateadores['Bateador'].str.replace('Á', 'A').str.replace('É', 'E').str.replace('Í', 'I').str.replace('Ó', 'O').str.replace('Ú', 'U')
bateadores['Bateador'] = bateadores['Bateador'].str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')

## Separar en 2 dataframes uno para local y otro para visitante

In [ ]:
homeAtBats = df[df['Top/Bottom'] == 'Bottom'].copy()
awayAtBats = df[df['Top/Bottom'] == 'Top'].copy()

homePitchers = pd.merge(
    awayAtBats['Pitcher'], pitchers,
    on='Pitcher'
).drop_duplicates().reset_index(drop=True)
awayPitchers = pd.merge(
    homeAtBats['Pitcher'], pitchers,
    on='Pitcher'
).drop_duplicates()

homeBateadores = pd.merge(
    homeAtBats['Batter'], bateadores,
    left_on='Batter', right_on='Bateador'
).drop_duplicates().reset_index(drop=True)
awayBateadores = pd.merge(
    awayAtBats['Batter'], bateadores,
    left_on='Batter', right_on='Bateador'
).drop_duplicates().reset_index(drop=True)

## Diablos

### Lineup

In [ ]:
lineUpDiablos = homeBateadores[['Bateador', 'Position', 'seasonAVG', 'seasonOPS']].rename(
    columns={'Bateador': 'Nombre', 'Position': 'Posición', 'seasonAVG': 'AVG', 'seasonOPS': 'OPS'}
)
#lineUpDiablos.to_csv('../reporte/csv/lineUp/lineUpDiablos.csv', index=False, encoding='utf-8-sig')
#lineUpDiablos

### Pitchers

In [ ]:
ordenColumnas = ['Nombre', 'Entrada de inicio', 'Entradas lanzadas', 'ERA']
pitchersDiablos = pd.merge(
    homePitchers[['Pitcher', 'InningsPitch', 'seasonERA']], awayAtBats.groupby('Pitcher')['Inning'].min(),
    on='Pitcher'
).rename(
    columns={'Pitcher': 'Nombre', 'Inning': 'Entrada de inicio', 'InningsPitch':'Entradas lanzadas', 'seasonERA': 'ERA'}
)[ordenColumnas]
#pitchersDiablos.to_csv('../reporte/csv/pitchers/pitchersDiablos.csv', index=False, encoding='utf-8-sig')
#pitchersDiablos

### Analisis por pitcher

#### Línea de pitcheo

In [ ]:
lineaPitcheoDiablos = pd.merge(
    homePitchers[['Team', 'Pitcher', 'BattersFaced', 'InningsPitch', 'runs', 'earnedRuns']],
    awayAtBats.groupby(['Pitcher', 'BatterSide']).agg(
        NoLanzamientos=pd.NamedAgg(column='PitchNo', aggfunc='count'),
        startInning=pd.NamedAgg(column='Inning', aggfunc='min'),
        Strikes=pd.NamedAgg(column='PitchCall', aggfunc=lambda x: np.where(
            (x.str.startswith('Strike')) | (x.str.startswith('FoulBall')) | (x.str.startswith('InPlay')), 1, 0).sum()),
        SwingStrikes=pd.NamedAgg(column='PitchCall', aggfunc=lambda x: np.where(x == 'StrikeSwinging', 1, 0).sum()),
        Bolas=pd.NamedAgg(column='PitchCall', aggfunc=lambda x: np.where(x.str.startswith('Ball'), 1, 0).sum()),
        Ponches=pd.NamedAgg(column='KorBB', aggfunc=lambda x: np.where(x == 'Strikeout', 1, 0).sum()),
        Walk=pd.NamedAgg(column='KorBB', aggfunc=lambda x: np.where(x == 'Walk', 1, 0).sum())
    ).reset_index(), on='Pitcher'
).rename(
    columns={
        'Pitcher': 'Nombre', 'NoLanzamientos': 'No lanzamientos', 'BattersFaced': 'Bateadores enfrentados',
        'startInning': 'Entrada de inicio', 'InningsPitch': 'Entradas lanzadas', 'SwingStrikes': 'Strikes con swing',
        'Walk': 'Bases por bolas', 'runs': 'Carreras', 'earnedRuns': 'Carreras limpias'
    }
)
#lineaPitcheoDiablos

In [ ]:
temp=lineaPitcheoDiablos.groupby('Nombre').agg({
        'No lanzamientos': 'sum', 
        'Bateadores enfrentados': 'min',
        'Entrada de inicio': 'min',
        'Entradas lanzadas': 'min',
        'Strikes': 'sum',
        'Strikes con swing': 'sum',
        'Bolas': 'sum',
        'Ponches': 'sum',
        'Bases por bolas': 'sum',
        'Carreras': 'min',
        'Carreras limpias': 'min'
    })

#### Distribución de pitcheos

In [ ]:
distribucionLanzamientosDaiablos = pd.merge(
    lineaPitcheoDiablos[['Team', 'Nombre', 'BatterSide', 'No lanzamientos', 'Strikes', 'Strikes con swing', 'Bolas', 'Ponches', 'Bases por bolas']],
    awayAtBats.groupby(['Pitcher', 'BatterSide', 'TaggedPitchType']).agg(
        Uso=pd.NamedAgg(column='PitchNo', aggfunc='count'),
        MaxSpeed= pd.NamedAgg(column='RelSpeed', aggfunc=lambda x: round(x.max(), 1)),
        AvgSpeed=pd.NamedAgg(column='RelSpeed', aggfunc=lambda x:round(x.mean(), 1)),
        StrikesPorTipo=pd.NamedAgg(column='PitchCall', aggfunc=lambda x: np.where(
            (x.str.startswith('Strike')) | (x.str.startswith('FoulBall')) | (x.str.startswith('InPlay')), 1, 0).sum()),
        SwingStrikesPorTipo=pd.NamedAgg(column='PitchCall', aggfunc=lambda x: np.where(x == 'StrikeSwinging', 1, 0).sum()),
        BolasPorTipo=pd.NamedAgg(column='PitchCall', aggfunc=lambda x: np.where(x.str.startswith('Ball'), 1, 0).sum()),
        PonchesPorTipo=pd.NamedAgg(column='KorBB', aggfunc=lambda x: np.where(x == 'Strikeout', 1, 0).sum())
    ).reset_index(), left_on=['Nombre', 'BatterSide'], right_on=['Pitcher', 'BatterSide']
).rename(columns={'TaggedPitchType': 'Tipo lanzamiento', 'MaxSpeed': 'Velocidad maxima', 'AvgSpeed': 'Velocidad promedio'})
distribucionLanzamientosDaiablos=pd.concat(
    [
    distribucionLanzamientosDaiablos, 
    distribucionLanzamientosDaiablos.groupby(['Team', 'Nombre', 'Tipo lanzamiento']).agg({
        'BatterSide': lambda x: 'Total',
        'No lanzamientos': 'sum',
        'Strikes': 'sum',
        'Strikes con swing': 'sum',
        'Bolas': 'sum',
        'Ponches': 'sum',
        'Bases por bolas': 'sum',
        'Uso': 'sum',
        'Velocidad maxima': 'max',
        'Velocidad promedio': 'mean',
        'StrikesPorTipo': 'sum',
        'SwingStrikesPorTipo': 'sum',
        'BolasPorTipo': 'sum',
        'PonchesPorTipo': 'sum'
    }).reset_index()
    ])
distribucionLanzamientosDaiablos['Porcentaje de uso'] = round(
    (100*distribucionLanzamientosDaiablos['Uso'])/distribucionLanzamientosDaiablos['No lanzamientos'], 1)
distribucionLanzamientosDaiablos['Strikes %'] = round(
    (100*distribucionLanzamientosDaiablos['StrikesPorTipo'])/distribucionLanzamientosDaiablos['Strikes'], 1)
distribucionLanzamientosDaiablos['Swing Strikes %'] = round(
    (100*distribucionLanzamientosDaiablos['SwingStrikesPorTipo'])/distribucionLanzamientosDaiablos['Strikes con swing'], 1)
distribucionLanzamientosDaiablos['Bolas %'] = round(
    (100*distribucionLanzamientosDaiablos['BolasPorTipo'])/distribucionLanzamientosDaiablos['Bolas'], 1)
distribucionLanzamientosDaiablos['Ponches %'] = round(
    (100*distribucionLanzamientosDaiablos['PonchesPorTipo'])/distribucionLanzamientosDaiablos['Ponches'], 1)
distribucionLanzamientosDaiablos = distribucionLanzamientosDaiablos.drop(columns=['Pitcher']).fillna(0)
#distribucionLanzamientosDaiablos

#### Situaciones de riesgo

In [ ]:
resultadoSituacionRiesgoDiablos = awayAtBats[(awayAtBats['Balls'] == 0) & (awayAtBats['Strikes'] == 2)][['Inning', 'PAofInning', 'Pitcher', 'PAResult']]
resultadoSituacionRiesgoDiablos = resultadoSituacionRiesgoDiablos.groupby(['Inning', 'PAofInning']).last().reset_index()

resultadoSituacionRiesgoDiablos = pd.get_dummies(resultadoSituacionRiesgoDiablos, columns=['PAResult'], dtype=int)
resultadoSituacionRiesgoDiablos['Total'] = resultadoSituacionRiesgoDiablos.groupby(['Pitcher'])['Pitcher'].transform('count')
resultadoSituacionRiesgoDiablos = resultadoSituacionRiesgoDiablos.groupby('Pitcher').apply(lambda x: round(x.mean()*100, 1)).reset_index()
resultadoSituacionRiesgoDiablos['Total'] = resultadoSituacionRiesgoDiablos['Total']/100
newColumns = [x.replace('PAResult_', '') for x in resultadoSituacionRiesgoDiablos.columns]
resultadoSituacionRiesgoDiablos = resultadoSituacionRiesgoDiablos.rename(columns=dict(zip(resultadoSituacionRiesgoDiablos.columns, newColumns)))
#resultadoSituacionRiesgoDiablos

#### Situaciones de ventaja

In [ ]:
resultadoSituacionVentajaDiablos = awayAtBats[(awayAtBats['Balls'] == 0) & (awayAtBats['Strikes'] == 2)][['Inning', 'PAofInning', 'Pitcher', 'PAResult']]
resultadoSituacionVentajaDiablos = resultadoSituacionVentajaDiablos.groupby(['Inning', 'PAofInning']).last().reset_index()

resultadoSituacionVentajaDiablos = pd.get_dummies(resultadoSituacionVentajaDiablos, columns=['PAResult'], dtype=int)
resultadoSituacionVentajaDiablos['Total'] = resultadoSituacionVentajaDiablos.groupby(['Pitcher'])['Pitcher'].transform('count')
resultadoSituacionVentajaDiablos = resultadoSituacionVentajaDiablos.groupby('Pitcher').apply(lambda x: round(x.mean()*100, 1)).reset_index()
resultadoSituacionVentajaDiablos['Total'] = resultadoSituacionVentajaDiablos['Total']/100
newColumns = [x.replace('PAResult_', '') for x in resultadoSituacionVentajaDiablos.columns]
resultadoSituacionVentajaDiablos = resultadoSituacionVentajaDiablos.rename(columns=dict(zip(resultadoSituacionVentajaDiablos.columns, newColumns)))
#resultadoSituacionVentajaDiablos

#### Secuencia de pitcheos

In [ ]:
secuenciaPitcheosDiablos = pd.merge(
    awayAtBats[awayAtBats['PitchofPA'] <= 4][
        ['Inning', 'PAofInning', 'PitchofPA', 'Pitcher', 'Batter', 'BatterSide', 'RunDif', 
         'TaggedPitchType', 'PitchZone', 'PAResult']],
    bateadores[['Bateador', 'seasonAVG']].rename(columns={'Bateador': 'Batter'}),
    on='Batter', how='left'
)

secuenciaPitcheosDiablos['PitchInfo'] = secuenciaPitcheosDiablos['TaggedPitchType'] + ' ' + secuenciaPitcheosDiablos['PitchZone'].astype(str)

secuenciaPitcheosDiablos = secuenciaPitcheosDiablos.pivot_table(
    index=['Inning', 'PAofInning', 'Pitcher', 'Batter', 'BatterSide', 'seasonAVG', 'RunDif', 'PAResult'],
    columns='PitchofPA',
    values='PitchInfo',
    aggfunc='last'
).sort_values(by=['Inning', 'PAofInning']).reset_index().drop(columns=['Inning', 'PAofInning'])
secuenciaPitcheosDiablos.columns = ['Pitcher', 'Batter', 'BatterSide', 'seasonAVG', 'RunDif', 'PAResult', '1', '2', '3', '4']
secuenciaPitcheosDiablos = secuenciaPitcheosDiablos[
    ['Pitcher', 'Batter', 'BatterSide', 'seasonAVG', 'RunDif', '1', '2', '3', '4', 'PAResult']
].sort_values(by=['BatterSide', 'seasonAVG']).reset_index(drop=True)

#### Wilmer Font

##### Línea de pitcheo

In [ ]:
# Seleccionar solo el pitcher Wilmer Font
lineaPitcheoDiablosWilmerFont=lineaPitcheoDiablos[lineaPitcheoDiablos['Nombre'] == 'Wilmer Font'].agg({
    'No lanzamientos': 'sum', 
    'Bateadores enfrentados': 'min',
    'Entrada de inicio': 'min',
    'Entradas lanzadas': 'min',
    'Strikes': 'sum',
    'Strikes con swing': 'sum',
    'Bolas': 'sum',
    'Ponches': 'sum',
    'Bases por bolas': 'sum',
    'Carreras': 'min',
    'Carreras limpias': 'min'
})
#lineaPitcheoDiablosWilmerFont.to_frame().T.to_csv('../reporte/csv/lineaPitcheo/lineaWilmerFont.csv', index=False, encoding='utf-8-sig')

##### Distribucion de pitcheo total

In [ ]:
columnas = ['Tipo lanzamiento', 'Uso', 'Porcentaje de uso', 'Velocidad maxima', 'Velocidad promedio', 
            'Strikes %', 'Swing Strikes %', 'Bolas %', 'Ponches %']
distribucionWilmerFont = distribucionLanzamientosDaiablos[
    (distribucionLanzamientosDaiablos['Nombre'] == 'Wilmer Font') &
    (distribucionLanzamientosDaiablos['BatterSide']=='Total')
].sort_values(by='Uso', ascending=False).reset_index(drop=True)
#distribucionWilmerFont[columnas].to_csv('../reporte/csv/distribucionPitcheo/total/distribucionWilmerFont.csv', index=False, encoding='utf-8-sig')
#distribucionWilmerFont

In [ ]:
ruta = '../reporte/img/distribucionPitcheo/total/distribucion{0}WilmerFont.png'

titulo = 'Distribución de lanzamientos de Wilmer Font'
distribucionWilmerFontImg = graficarDistribucionLanzamientos(
    dfDist=distribucionWilmerFont.rename(columns={'Porcentaje de uso': 'Porcentaje', 'Uso': 'Total'}),
    dfPos=awayAtBats[awayAtBats['Pitcher'] == 'Wilmer Font'],
    titulo=titulo
)
distribucionWilmerFontImg.savefig(ruta.format(''), bbox_inches='tight', dpi=300)

titulo = 'Distribución de strikes de Wilmer Font'
distribucionStrikesWilmerFontImg = graficarDistribucionLanzamientos(
    dfDist=distribucionWilmerFont.rename(columns={'Strikes %': 'Porcentaje', 'StrikesPorTipo': 'Total'}),
    dfPos=awayAtBats[(awayAtBats['Pitcher'] == 'Wilmer Font') & 
                     ((awayAtBats['PitchCall'].str.startswith('Strike')) |
                      (awayAtBats['PitchCall'].str.startswith('FoulBall')) |
                      (awayAtBats['PitchCall'].str.startswith('InPlay')))],
    titulo=titulo
)
distribucionStrikesWilmerFontImg.savefig(ruta.format('Strikes'), bbox_inches='tight', dpi=300)

titulo = 'Distribución de ponches de Wilmer Font'
distribucionPonchesWilmerFontImg = graficarDistribucionLanzamientos(
    dfDist=distribucionWilmerFont.rename(columns={'Ponches %': 'Porcentaje', 'PonchesPorTipo': 'Total'}),
    dfPos=awayAtBats[(awayAtBats['Pitcher'] == 'Wilmer Font') & 
                     (awayAtBats['KorBB'].str.startswith('Strikeout'))],
    titulo=titulo
)
distribucionPonchesWilmerFontImg.savefig(ruta.format('Ponches'), bbox_inches='tight', dpi=300)

titulo = 'Distribución de bolas de Wilmer Font'
distribucionBolasWilmerFontImg = graficarDistribucionLanzamientos(
    dfDist=distribucionWilmerFont.rename(columns={'Bolas %': 'Porcentaje', 'BolasPorTipo': 'Total'}),
    dfPos=awayAtBats[(awayAtBats['Pitcher'] == 'Wilmer Font') & 
                     (awayAtBats['PitchCall'].str.startswith('Ball'))],
    titulo=titulo
)
distribucionBolasWilmerFontImg.savefig(ruta.format('Bolas'), bbox_inches='tight', dpi=300)

##### Distribucion de pitcheo vs derechos

In [ ]:
columnas = ['Tipo lanzamiento', 'Uso', 'Porcentaje de uso', 'Velocidad maxima', 'Velocidad promedio', 
            'Strikes %', 'Swing Strikes %', 'Bolas %', 'Ponches %']
distribucionVsRWilmerFont = distribucionLanzamientosDaiablos[
    (distribucionLanzamientosDaiablos['Nombre'] == 'Wilmer Font') &
    (distribucionLanzamientosDaiablos['BatterSide']=='Right')
].sort_values(by='Uso', ascending=False).reset_index(drop=True)
#distribucionVsRWilmerFont[columnas].to_csv('../reporte/csv/distribucionPitcheo/right/distribucionVsDerechosWilmerFont.csv', index=False, encoding='utf-8-sig')
#distribucionVsRWilmerFont

In [ ]:
ruta = '../reporte/img/distribucionPitcheo/right/distribucionVsR{0}WilmerFont.png'

titulo = 'Distribución de lanzamientos de Wilmer Font vs bateadores derechos'
distribucionVsRWilmerFontImg = graficarDistribucionLanzamientos(
    dfDist=distribucionVsRWilmerFont.rename(columns={'Porcentaje de uso': 'Porcentaje', 'Uso': 'Total'}),
    dfPos=awayAtBats[(awayAtBats['Pitcher'] == 'Wilmer Font') &
                     (awayAtBats['BatterSide'] == 'Right')],
    titulo=titulo
)
#distribucionVsRWilmerFontImg.savefig(ruta.format(''), bbox_inches='tight', dpi=300)

titulo = 'Distribución de strikes de Wilmer Font vs bateadores derechos'
distribucionStrikesVsRWilmerFontImg = graficarDistribucionLanzamientos(
    dfDist=distribucionVsRWilmerFont.rename(columns={'Strikes %': 'Porcentaje', 'StrikesPorTipo': 'Total'}),
    dfPos=awayAtBats[(awayAtBats['Pitcher'] == 'Wilmer Font') &
                     (awayAtBats['BatterSide'] == 'Right') & 
                     ((awayAtBats['PitchCall'].str.startswith('Strike')) |
                      (awayAtBats['PitchCall'].str.startswith('FoulBall')) |
                      (awayAtBats['PitchCall'].str.startswith('InPlay')))],
    titulo=titulo
)
#distribucionStrikesVsRWilmerFontImg.savefig(ruta.format('Strikes'), bbox_inches='tight', dpi=300)

titulo = 'Distribución de ponches de Wilmer Font vs bateadores derechos'
distribucionPonchesVsRWilmerFontImg = graficarDistribucionLanzamientos(
    dfDist=distribucionVsRWilmerFont.rename(columns={'Ponches %': 'Porcentaje', 'PonchesPorTipo': 'Total'}),
    dfPos=awayAtBats[(awayAtBats['Pitcher'] == 'Wilmer Font')  &
                     (awayAtBats['BatterSide'] == 'Right')& 
                     (awayAtBats['KorBB'].str.startswith('Strikeout'))],
    titulo=titulo
)
#distribucionPonchesVsRWilmerFontImg.savefig(ruta.format('Ponches'), bbox_inches='tight', dpi=300)

titulo = 'Distribución de bolas de Wilmer Font vs bateadores derechos'
distribucionBolasVsRWilmerFontImg = graficarDistribucionLanzamientos(
    dfDist=distribucionVsRWilmerFont.rename(columns={'Bolas %': 'Porcentaje', 'BolasPorTipo': 'Total'}),
    dfPos=awayAtBats[(awayAtBats['Pitcher'] == 'Wilmer Font')  &
                     (awayAtBats['BatterSide'] == 'Right')& 
                     (awayAtBats['PitchCall'].str.startswith('Ball'))],
    titulo=titulo
)
#distribucionBolasVsRWilmerFontImg.savefig(ruta.format('Bolas'), bbox_inches='tight', dpi=300)

##### Distribucion de pitcheo vs izquierdos

In [ ]:
columnas = ['Tipo lanzamiento', 'Uso', 'Porcentaje de uso', 'Velocidad maxima', 'Velocidad promedio', 
            'Strikes %', 'Swing Strikes %', 'Bolas %', 'Ponches %']
distribucionVsLWilmerFont = distribucionLanzamientosDaiablos[
    (distribucionLanzamientosDaiablos['Nombre'] == 'Wilmer Font') &
    (distribucionLanzamientosDaiablos['BatterSide']=='Left')
].sort_values(by='Uso', ascending=False).reset_index(drop=True)
#distribucionVsLWilmerFont[columnas].to_csv('../reporte/csv/distribucionPitcheo/left/distribucionVsZurdosWilmerFont.csv', index=False, encoding='utf-8-sig')
#distribucionVsLWilmerFont

In [ ]:
ruta = '../reporte/img/distribucionPitcheo/left/distribucionVsL{0}WilmerFont.png'

titulo = 'Distribución de lanzamientos de Wilmer Font vs bateadores izquierdos'
distribucionVsLWilmerFontImg = graficarDistribucionLanzamientos(
    dfDist=distribucionVsLWilmerFont.rename(columns={'Porcentaje de uso': 'Porcentaje', 'Uso': 'Total'}),
    dfPos=awayAtBats[(awayAtBats['Pitcher'] == 'Wilmer Font') &
                     (awayAtBats['BatterSide'] == 'Left')],
    titulo=titulo
)
#distribucionVsLWilmerFontImg.savefig(ruta.format(''), bbox_inches='tight', dpi=300)

titulo = 'Distribución de strikes de Wilmer Font vs bateadores izquierdos'
distribucionStrikesVsLWilmerFontImg = graficarDistribucionLanzamientos(
    dfDist=distribucionVsLWilmerFont.rename(columns={'Strikes %': 'Porcentaje', 'StrikesPorTipo': 'Total'}),
    dfPos=awayAtBats[(awayAtBats['Pitcher'] == 'Wilmer Font') &
                     (awayAtBats['BatterSide'] == 'Left') & 
                     ((awayAtBats['PitchCall'].str.startswith('Strike')) |
                      (awayAtBats['PitchCall'].str.startswith('FoulBall')) |
                      (awayAtBats['PitchCall'].str.startswith('InPlay')))],
    titulo=titulo
)
#distribucionStrikesVsLWilmerFontImg.savefig(ruta.format('Strikes'), bbox_inches='tight', dpi=300)

titulo = 'Distribución de ponches de Wilmer Font vs bateadores izquierdos'
distribucionPonchesVsLWilmerFontImg = graficarDistribucionLanzamientos(
    dfDist=distribucionVsLWilmerFont.rename(columns={'Ponches %': 'Porcentaje', 'PonchesPorTipo': 'Total'}),
    dfPos=awayAtBats[(awayAtBats['Pitcher'] == 'Wilmer Font')  &
                     (awayAtBats['BatterSide'] == 'Left')& 
                     (awayAtBats['KorBB'].str.startswith('Strikeout'))],
    titulo=titulo
)
#distribucionPonchesVsLWilmerFontImg.savefig(ruta.format('Ponches'), bbox_inches='tight', dpi=300)

titulo = 'Distribución de bolas de Wilmer Font vs bateadores izquierdos'
distribucionBolasVsLWilmerFontImg = graficarDistribucionLanzamientos(
    dfDist=distribucionVsLWilmerFont.rename(columns={'Bolas %': 'Porcentaje', 'BolasPorTipo': 'Total'}),
    dfPos=awayAtBats[(awayAtBats['Pitcher'] == 'Wilmer Font')  &
                     (awayAtBats['BatterSide'] == 'Left')& 
                     (awayAtBats['PitchCall'].str.startswith('Ball'))],
    titulo=titulo
)
#distribucionBolasVsLWilmerFontImg.savefig(ruta.format('Bolas'), bbox_inches='tight', dpi=300)

##### Situaciones de riesgo

In [ ]:
situacionRiesgoWilmerFont = resultadoSituacionRiesgoDiablos[resultadoSituacionRiesgoDiablos['Pitcher'] == 'Wilmer Font'].reset_index(drop=True)
total = int(situacionRiesgoWilmerFont['Total'].to_numpy()[0])
situacionRiesgoWilmerFont = situacionRiesgoWilmerFont.drop(columns=['Total', 'Pitcher']).T.rename(columns={0: 'Porcentaje'})
situacionRiesgoWilmerFont = situacionRiesgoWilmerFont[situacionRiesgoWilmerFont['Porcentaje'] > 0].reset_index()

In [ ]:
situacionRiesgoWilmerFontIMG = graficarSituacionPitcher(situacionRiesgoWilmerFont, total, 'Situaciones de riesgo de Wilmer Font')
#situacionRiesgoWilmerFontIMG.savefig('../reporte/img/situacionRiesgo/situacionRiesgoWilmerFont.png', bbox_inches='tight', dpi=300)

##### Situaciones de ventaja

In [ ]:
situacionVentajaWilmerFont = resultadoSituacionVentajaDiablos[resultadoSituacionVentajaDiablos['Pitcher'] == 'Wilmer Font'].reset_index(drop=True)
total = int(situacionVentajaWilmerFont['Total'].to_numpy()[0])
situacionVentajaWilmerFont = situacionVentajaWilmerFont.drop(columns=['Total', 'Pitcher']).T.rename(columns={0: 'Porcentaje'})
situacionVentajaWilmerFont = situacionVentajaWilmerFont[situacionVentajaWilmerFont['Porcentaje'] > 0].reset_index()

In [ ]:
situacionVentajaWilmerFontIMG = graficarSituacionPitcher(situacionRiesgoWilmerFont, total, 'Situaciones de ventaja de Wilmer Font')
#situacionVentajaWilmerFontIMG.savefig('../reporte/img/situacionVentaja/situacionVentajaWilmerFont.png', bbox_inches='tight', dpi=300)

##### Secuencia de pitcheos

In [ ]:
secuenciaPitcheosWilmerFont = secuenciaPitcheosDiablos[secuenciaPitcheosDiablos['Pitcher'] == 'Wilmer Font']

secuenciaPitcheosVsLWilmerFont = secuenciaPitcheosWilmerFont[secuenciaPitcheosWilmerFont['BatterSide'] == 'Left'].reset_index(drop=True)
#secuenciaPitcheosVsLWilmerFont.to_csv('../reporte/csv/secuenciaPitcheos/left/secuenciaPitcheosVsLWilmerFont.csv', index=False, encoding='utf-8-sig')
secuenciaPitcheosVsRWilmerFont = secuenciaPitcheosWilmerFont[secuenciaPitcheosWilmerFont['BatterSide'] == 'Right'].reset_index(drop=True)
#secuenciaPitcheosVsRWilmerFont.to_csv('../reporte/csv/secuenciaPitcheos/right/secuenciaPitcheosVsRWilmerFont.csv', index=False, encoding='utf-8-sig')